# コード

## ライブラリのインポート

In [ ]:
from qiskit import *
from qiskit.circuit.library import UnitaryGate
from qiskit_aer import Aer
from numpy import sqrt

## 回路作成

In [ ]:
qc = QuantumCircuit(2,2)

qc.h(0)
qc.ch(0,1)
qc.h(0)
qc.measure([0],[0])

qc.draw('mpl')

## 回路実行

In [ ]:
backend = Aer.get_backend("qasm_simulator")
qc_transpiled = transpile(qc, backend, basis_gates=['u1', 'u2', 'u3', 'cx', 'id'])
result = backend.run(qc_transpiled).result()
counts = result.get_counts()
sum_0 = sum(value for key, value in counts.items() if key[1] == '0')
sum_1 = sum(value for key, value in counts.items() if key[1] == '1')
print(f"{{'0': {sum_0}, '1': {sum_1}}}")

# 解説

以下のような、制御アダマールゲートを使用したアダマールテストを考えます。

![スクリーンショット 2024-06-12 12.10.21.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/2607452/8c2e2ab0-abc7-6dea-2219-63740078cb20.png)

また、初期状態は以下とします。
$q[0] = |0\rangle$
$q[1] = |0\rangle$

ここで、回路を層に分けて計算します。

![スクリーンショット 2024-06-12 12.10.21 2.png](https://qiita-image-store.s3.ap-northeast-1.amazonaws.com/0/2607452/fd6b98c8-c6a4-2b03-c703-d2c8c3bb66ba.png)

並列回路におけるゲートの合成は、テンソル積で計算できます。

$U_1 = H \otimes I = \frac{1}{\sqrt2}\left(\begin{matrix} 1&0&1&0 \\ 0&1&0&1 \\ 1&0&-1&0 \\ 0&1&0&-1 \end{matrix}\right)$

$U_2 = CH_{(0,1)} = \left(\begin{matrix} 1&0&0&0 \\ 0&1&0&0 \\ 0&0& \frac{1}{\sqrt{2}} &\frac{1}{\sqrt{2}} \\ 0&0&\frac{1}{\sqrt{2}}&-\frac{1}{\sqrt{2}} \end{matrix}\right)$

$U_3 = H \otimes I = \frac{1}{\sqrt2}\left(\begin{matrix} 1&0&1&0 \\ 0&1&0&1 \\ 1&0&-1&0 \\ 0&1&0&-1 \end{matrix}\right)$

ここで、合成されたゲートを適用していきます。

$U_3U_2U_1|00\rangle = U_3U_2\frac{1}{\sqrt2}\left(\begin{matrix} 1&0&1&0 \\ 0&1&0&1 \\ 1&0&-1&0 \\ 0&1&0&-1 \end{matrix}\right) \left(\begin{matrix} 1\\0\\0\\0 \end{matrix}\right)$

$=U_3U_2\frac{1}{\sqrt2}\left(\begin{matrix} 1\\0\\1\\0 \end{matrix}\right)$

$=U_3\left(\begin{matrix} 1&0&0&0 \\ 0&1&0&0 \\ 0&0& \frac{1}{\sqrt{2}} &\frac{1}{\sqrt{2}} \\ 0&0&\frac{1}{\sqrt{2}}&-\frac{1}{\sqrt{2}} \end{matrix}\right)\frac{1}{\sqrt2}\left(\begin{matrix} 1\\0\\1\\0 \end{matrix}\right)$

$= U_3\frac{1}{\sqrt2}\left(\begin{matrix} 1\\0\\\frac{1}{\sqrt2}\\\frac{1}{\sqrt2} \end{matrix}\right)$

$= \frac{1}{\sqrt2}\left(\begin{matrix} 1&0&1&0 \\ 0&1&0&1 \\ 1&0&-1&0 \\ 0&1&0&-1 \end{matrix}\right) \frac{1}{\sqrt2}\left(\begin{matrix} 1\\0\\\frac{1}{\sqrt2}\\\frac{1}{\sqrt2} \end{matrix}\right)$

$= \frac{1}{2} \left(\begin{matrix} \frac{\sqrt2+1}{\sqrt2}\\ \frac{1}{\sqrt2}\\ \frac{\sqrt2-1}{\sqrt2}\\ -\frac{1}{\sqrt2}\\  \end{matrix}\right)$

$= \left(\frac{\sqrt2+1}{2\sqrt2}|00\rangle + \frac{1}{2\sqrt2}|01\rangle + \frac{\sqrt2-1}{2\sqrt2}|10\rangle - \frac{1}{2\sqrt2}|11\rangle\right)$

1量子目の測定結果が0となる確率$p_0$は、

$p_0 = \left| \frac{\sqrt2+1}{2\sqrt2} \right|^2 + \left| \frac{1}{2\sqrt2} \right|^2 = \frac{4+2\sqrt2}{8}$

2量子目の測定結果が0となる確率$p_1$は、

$p_1 = \left| \frac{\sqrt2-1}{2\sqrt2} \right|^2 + \left| \frac{1}{2\sqrt2} \right|^2 = \frac{4-2\sqrt2}{8}$

つまり、初期状態が$q[0]=|0\rangle$、$q[1]=|0\rangle$のアダマールテストにおいて、制御アダマールゲートを使用した時、1量子目の測定は

$p_0 = \frac{4+2\sqrt2}{8} \approx 0.85$

$p_1 = \frac{4-2\sqrt2}{8} \approx 0.15$

となります。